In [1]:
from dataloader_classes import Lung_Train_Dataset, Lung_Test_Dataset, Lung_Val_Dataset
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms

In [2]:
ld_train = Lung_Train_Dataset()
ld_train.describe()
print(len(ld_train))
im, class_oh = ld_train[5215]
print(im.shape)
print(im)
print(class_oh)

This is the training dataset of the Lung Dataset used for the Small Project in the 50.039 Deep Learning class in Feb-March 2021. 
It contains a total of 5216 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - train_normal, in folder ./dataset/train/normal/: 1341 images.
 - train_infected(non_covid), in folder ./dataset/train/infected/non-covid/: 2530 images.
 - train_infected(covid), in folder ./dataset/train/infected/covid/: 1345 images.

5216
torch.Size([1, 150, 150])
tensor([[[0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])
tensor([0., 0., 1.])


In [3]:
ld_test = Lung_Test_Dataset()
ld_test.describe()
print(len(ld_test))
im, class_oh = ld_test[476]
print(im.shape)
print(im)
print(class_oh)

This is the test dataset of the Lung Dataset used for the Small Project in the 50.039 Deep Learning class in Feb-March 2021. 
It contains a total of 615 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - test_normal, in folder ./dataset/test/normal/: 234 images.
 - test_infected(non_covid), in folder ./dataset/test/infected/non-covid/: 242 images.
 - test_infected(covid), in folder ./dataset/test/infected/covid/: 139 images.

615
torch.Size([1, 150, 150])
tensor([[[0.2588, 0.3725, 0.4314,  ..., 0.1176, 0.1255, 0.1333],
         [0.3686, 0.4196, 0.4549,  ..., 0.1137, 0.1216, 0.1255],
         [0.4392, 0.4275, 0.4510,  ..., 0.0941, 0.0941, 0.0941],
         ...,
         [0.1647, 0.1765, 0.1412,  ..., 0.1569, 0.1569, 0.1569],
         [0.1647, 0.1725, 0.1373,  ..., 0.1569, 0.1569, 0.1569],
         [0.1608, 0.1725, 0.1373,  ..., 0.1569, 0.1569, 0.1569]]])
tensor([0., 0., 1.])


In [4]:
ld_val = Lung_Val_Dataset()
ld_val.describe()
print(len(ld_val))
im, class_oh = ld_val[15]
print(im.shape)
print(im)
print(class_oh)

This is the validation dataset of the Lung Dataset used for the Small Project in the 50.039 Deep Learning class in Feb-March 2021. 
It contains a total of 25 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - val_normal, in folder ./dataset/val/normal/: 8 images.
 - val_infected(non_covid), in folder ./dataset/val/infected/non-covid: 8 images.
 - val_infected(covid), in folder ./dataset/val/infected/covid: 9 images.

25
torch.Size([1, 150, 150])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
         [0.0078, 0.0039, 0.0039,  ..., 0.0078, 0.0118, 0.0157],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])
tensor([0., 1., 0.])


In [5]:
bs_val = 8

In [6]:
# creating dataloader object
train_loader = DataLoader(ld_train, batch_size = bs_val, shuffle = True)
print(train_loader)
test_loader = DataLoader(ld_test, batch_size = bs_val, shuffle = True)
print(test_loader)
val_loader = DataLoader(ld_val, batch_size = bs_val, shuffle = True)
print(val_loader)

In [7]:
# Typical mini-batch for loop on dataloader (train)
for k, v in enumerate(train_loader):
    print("-----")
    print(k)
    print(v[0])
    print(v[1])
    # Forced stop
    break
    #assert False, "Forced stop after one iteration of the for loop"

-----
0
tensor([[[[0.2196, 0.2784, 0.2863,  ..., 0.1608, 0.1451, 0.1333],
          [0.2196, 0.2667, 0.2745,  ..., 0.1529, 0.1333, 0.1255],
          [0.2235, 0.2510, 0.2667,  ..., 0.1451, 0.1255, 0.1137],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0078, 0.0549, 0.1686,  ..., 0.0353, 0.0078, 0.0000],
          [0.0235, 0.0588, 0.1529,  ..., 0.0353, 0.0039, 0.0000],
          [0.0235, 0.0627, 0.1529,  ..., 0.0353, 0.0000, 0.0000]]],


        [[[0.0588, 0.0902, 0.1333,  ..., 0.0000, 0.0000, 0.0000],
          [0.0471, 0.0784, 0.1255,  ..., 0.0000, 0.0000, 0.0000],
          [0.0353, 0.0706, 0.1

Testing of the dataloader

In [8]:
# A simple mode
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Conv2D: 1 input channel, 8 output channels, 3 by 3 kernel, stride of 1.
        self.conv1 = nn.Conv2d(1, 4, 3, 1)
        self.fc1 = nn.Linear(87616, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        output = F.log_softmax(x, dim = 1)
        return output

In [9]:
# Create model
model = Net()
# Try model on one mini-batch
for batch_idx, (images_data, target_labels) in enumerate(train_loader):
    predicted_labels = model(images_data)
    print(predicted_labels)
    print(target_labels)
    # Forced stop
    break
    #assert False, "Forced stop after one iteration of the mini-batch for loop"

tensor([[-0.6467, -0.7419],
        [-0.6536, -0.7344],
        [-0.6363, -0.7534],
        [-0.6019, -0.7936],
        [-0.5785, -0.8227],
        [-0.5598, -0.8471],
        [-0.5923, -0.8053],
        [-0.6432, -0.7457]], grad_fn=<LogSoftmaxBackward>)
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]])
